In [19]:
from utils import read_images, return_stacks_per_position_id, make_isotropic, extract_nuclei_labels, measure_intensity
from pathlib import Path
from tqdm import tqdm
import pandas as pd

In [20]:
# Define your data directory (folder containing the subfolders storing your images)
data_folder = Path("./raw_data")

# Define the scaling in x, y and z.
scaling_x_um = 0.342
scaling_y_um = 0.342
scaling_z_um = 0.663

# Define the analysis parameters
top_hat_radius = 5
gaussian_sigma = 2
voronoi_otsu_spot_sigma = 10
voronoi_otsu_outline_sigma = 1
closing_labels_radius = 5
erosion_labels_radius = 3

In [21]:
# Initialize an empty list to store subfolder names
subfolder_list = []

# Iterate over subdirectories in the parent folder
for subfolder in data_folder.iterdir():
    if subfolder.is_dir():
        subfolder_list.append(subfolder.name)

# Initialize an empty list to store per folder_id dataframes
folder_dataframes = []

for folder in tqdm(subfolder_list):
    
    # Build the path containing the input images
    data_path = data_folder.joinpath(folder)
    
    # Add the intermediate crop subdirectory
    data_path = data_path.joinpath('Crop')
    
    # Retrieve the paths of all the images within data_path (ignoring ch02, brightfield channel)
    images_per_position = read_images(data_path)

    # Initialize an empty list to store per position_id dataframes
    dataframes = []

    for position_id, files in images_per_position.items():
        
        # Retrieve the paths of all the images within data_path (ignoring ch02, brightfield channel)
        images_per_position = read_images(data_path)

        # Generate the ch00_stack (nuclei) and ch01_stack (protein of interest) from the individual slices 
        ch00_stack, ch01_stack = return_stacks_per_position_id(images_per_position, position_id)

        # Extract the nuclei labels from the ch00_stack
        nuclei_labels = extract_nuclei_labels(ch00_stack, 
                                scaling_x_um, 
                                scaling_y_um, 
                                scaling_z_um, 
                                top_hat_radius, 
                                gaussian_sigma, 
                                voronoi_otsu_spot_sigma,
                                voronoi_otsu_outline_sigma,
                                closing_labels_radius,
                                erosion_labels_radius)

        # Rescale the ch01_stack (protein of interest) to make data isotropic in order to include it as intensity_image 
        marker_resampled = make_isotropic(ch01_stack, scaling_x_um, scaling_y_um, scaling_z_um)

        # Extract intensity stats on a per nucleus basis
        df = measure_intensity(nuclei_labels, marker_resampled)

        # Insert position_id into the dataframe
        df.insert(0, 'position_id', position_id)

        # Append the per_position df to the dataframes list
        dataframes.append(df)

    # Concatenate all DataFrames in the list
    concatenated_df = pd.concat(dataframes, ignore_index=True)
    
    # Insert folder_id into the dataframe
    concatenated_df.insert(0, 'folder_id', folder)

    # Append the per folder_id dataframe to the folder_dataframes list
    folder_dataframes.append(concatenated_df)
    
final_df = pd.concat(folder_dataframes, ignore_index=True)

100%|██████████| 4/4 [00:30<00:00,  7.73s/it]


In [23]:
# Store the final results into a .csv file
final_df.to_csv('./results.csv')